In [1]:
import numpy as np
import csv
import pandas as pd
import tensorflow as tf

In [2]:
## 读取csv文件
data_train_origin = []
train = csv.reader(open('train.tsv', 'rt'))
for row in train:
    data_train_origin.append(row)


In [3]:
data_train_origin = data_train_origin[0:100]

In [4]:
## 数据的分割
data_train = []
for item in data_train_origin:
    if (len(item)>0):
        for j in range(1,len(item)):
            item[0]+=item[j]
    data_train.append(item[0].split('\t'))

    

In [5]:
## 删除标题
data_train.remove(data_train[0])

In [6]:
## 简单预处理
for i in range(len(data_train)):
    data_train[i][0] = int(data_train[i][0])
    data_train[i][1] = int(data_train[i][1])
    data_train[i][2] = data_train[i][2].replace('  ',' ')
    data_train[i][3] = int(data_train[i][3])
    

In [7]:
## 切分单词
tokenized_lines = []
for item in data_train:
    tokenized_lines.append(item[2].split(' '))

In [8]:
# 去除标点，替换大小写
punctuation = [",", ":", ";", ".", "'", '"', "’", "?", "/", "-", "+", "&", "(", ")"]
clean_tokenized = []
for item in tokenized_lines:
    tokens = []
    for token in item:
        token = token.lower()
        for punc in punctuation:
            token = token.replace(punc, "")
        tokens.append(token)
    clean_tokenized.append(tokens)

In [9]:
## 生成原始的词汇集合,剔除了只出现过一次的token
unique_tokens = []
single_tokens = []
for tokens in clean_tokenized:
    for token in tokens:
        if token not in single_tokens:
            single_tokens.append(token)
        elif token in single_tokens and token not in unique_tokens:
            unique_tokens.append(token)

In [10]:
## 统计每个词出现的频率
all_tokens = []
n = 0
for seq in clean_tokenized:
    for word in seq:
        all_tokens.append(word)
        if (word not in unique_tokens):
            n += 1

dic = {}
for token in unique_tokens:
    dic[token] = all_tokens.count(token)
dic['_UNK'] = n

In [11]:
## 构建哈夫曼树
# 定义节点类
class Node(object):
    def __init__(self,name=None,value=None):
        self._name=name
        self._value=value
        self._parent=None
        self._left=None
        self._right=None
    def get_name(self):
        return self._name
    def get_value(self):
        return self._value
    def get_parent(self):
        return self._parent
    def get_left(self):
        return self._left
    def get_right(self):
        return self._right

节点类包含的信息——名称、值、父节点、左子节点和右子节点。

In [12]:
#哈夫曼树类
class HuffmanTree(object):
    def __init__(self,frequency_dic):
        self.a=[Node(key,frequency_dic[key]) for key in frequency_dic.keys()] #根据输入的字符及其频数生成叶子节点
        self.leaves = []
        while len(self.a)!=1:    
            self.a.sort(key=lambda node:node._value,reverse=True)
            c=Node(value=(self.a[-1]._value+self.a[-2]._value))
            c._left=self.a[-1]
            c._right=self.a[-2]
            self.a[-1]._parent=c
            self.leaves.append(self.a[-1])
            self.a.pop(-1)
            self.a[-1]._parent=c
            self.leaves.append(self.a[-1])
            self.a.pop(-1)
            self.a.append(c)
        self.root=self.a[0]
        self.b=list(range(20))  #self.b用于保存每个叶子节点的Haffuman编码,range的值只需要不小于树的深度就行
        self.dic = {}
        
    #用递归的思想生成编码
    def pre(self,tree,length):
        node=tree
        if (not node):
            return
        elif node._name:
            print(node._name + '的编码为:')
            code = []
            for i in range(length):
                print(self.b[i])
                code.append(self.b[i])
            print('\n')
            self.dic[node._name] = code
            return
        self.b[length]=0
        self.pre(node._left,length+1)
        self.b[length]=1
        self.pre(node._right,length+1)
     #生成哈夫曼编码   
    def get_code(self):
        self.pre(self.root,0)
        self.a[0]._parent = None
        return self.leaves+self.a, self.dic
        
    

方法是从叶子开始，反向建立整个哈夫曼树，几个注意点：
1、 sort()函数的使用——key参数指定一个函数在排序之前调用，这里node代之之前的a中的元素，然后按照其value的属性进行排序；reverse是反转的意思，反向排序。
2、 pop()函数的使用——删除元素，当参数非指定元素而是数值的时候，删除从0到指定序号的元素。
3、 递归部分的设计
参考：
https://blog.csdn.net/lzq20115395/article/details/78906863

In [13]:
## 获取哈夫曼树
tree= HuffmanTree(dic)
leaves_list,leaves_code = tree.get_code()

of的编码为:
0
0
0


which的编码为:
0
0
1
0


would的编码为:
0
0
1
1
0
0
0


time的编码为:
0
0
1
1
0
0
1
0


hard的编码为:
0
0
1
1
0
0
1
1


ismail的编码为:
0
0
1
1
0
1


goose的编码为:
0
0
1
1
1


a的编码为:
0
1
0
0


none的编码为:
0
1
0
1
0


i的编码为:
0
1
0
1
1
0


entertaining的编码为:
0
1
0
1
1
1


the的编码为:
0
1
1
0


good的编码为:
0
1
1
1
0


demonstrating的编码为:
1
0
0
0
0
0


that的编码为:
1
0
0
0
0
1


merchant的编码为:
1
0
0
0
1
0


s的编码为:
1
0
0
0
1
1


amounts的编码为:
1
0
0
1
0


and的编码为:
1
0
0
1
1
0


adage的编码为:
1
0
0
1
1
1


to的编码为:
1
0
1
0
0


for的编码为:
1
0
1
0
1


through的编码为:
1
0
1
1
0
0
0
0


one的编码为:
1
0
1
1
0
0
0
1


_UNK的编码为:
1
0
1
1
0
0
1
0
0


sitting的编码为:
1
0
1
1
0
0
1
0
1


have的编码为:
1
0
1
1
0
0
1
1


introspective的编码为:
1
0
1
1
0
1


even的编码为:
1
0
1
1
1
0
0


series的编码为:
1
0
1
1
1
0
1


fans的编码为:
1
0
1
1
1
1
0


seeking的编码为:
1
0
1
1
1
1
1


what的编码为:
1
1
0
0
0
0


independent的编码为:
1
1
0
0
0
1
0


this的编码为:
1
1
0
0
0
1
1


much的编码为:
1
1
0
0
1


is的编码为:
1
1
0
1
0


gander的编码为:
1
1
0
1
1
0


occasionally的编码为:
1
1
0
1
1
1


some

In [14]:
## 将start和end加入到token中
unique_tokens.append('_STA')
unique_tokens.append('_END')

In [15]:
## 对词汇表进行编号，并生成反向词典
vocab = {}
for i in range(len(unique_tokens)):
    vocab[unique_tokens[i]] = i

reversed_vocab = {value:key for key,value in vocab.items()}


正向词典是通过词索引编号，反向词典是通过编号索引词汇，在抽取哈夫曼树枝的时候会用到

In [16]:
## 定义抽取词汇序号的函数
def get_index(word):
    if (word in unique_tokens):
        ind = vocab[word]
    else:
        ind = vocab['_UNK']
    return ind

In [17]:
## 表征文本中的每个词汇。
representations = []
words = []
for seq in clean_tokenized:
    lines = []
    lines_words = []
    for i in range(len(seq)):
        context = []
        context_words = []
        if (i == 0):
            context.append(get_index('_STA'))
            context.append(get_index('_STA'))
            context_words.append('_STA')
            context_words.append('_STA')
        elif (i == 1):
            context.append(get_index('_STA'))
            context.append(get_index(seq[i-1]))
            context_words.append('_STA')
            context_words.append(seq[i-1])
        else:
            context.append(get_index(seq[i-2]))
            context.append(get_index(seq[i-1]))
            context_words.append(seq[i-2])
            context_words.append(seq[i-1])
        
        if (i == len(seq)-1):
            context.append(get_index('_END'))
            context.append(get_index('_END'))
            context_words.append('_END')
            context_words.append('_END')
        elif (i == len(seq)-2):
            context.append(get_index(seq[i+1]))
            context.append(get_index('_END'))
            context_words.append(seq[i+1])
            context_words.append('_END')
        else:
            context.append(get_index(seq[i+1]))
            context.append(get_index(seq[i+2])) 
            context_words.append(seq[i+1])
            context_words.append(seq[i+2])
        lines.append([context,get_index(seq[i])])
        lines_words.append([context_words,seq[i]])
    representations.append(lines)
    words.append(lines_words)

选取2c = 4作为windows的小，前后各抽取两个词,称为Context(x),每个词汇都抽成[context(x),x]的形式

In [18]:
## 随机抽取元素进行训练，适用于SDG
import random
def random_sample(representations,words):
    index = random.sample(range(len(representations)),1)[0]
    sub_index = random.sample(range(len(representations[index])),1)[0]
    return representations[index][sub_index][0],representations[index][sub_index][1],words[index][sub_index][0],words[index][sub_index][1]

In [19]:
## 初始化参数，对应每个节点向上的树枝定义W和b
W = []
b = []
for j in range(len(leaves_list)-1):
    W.append(np.random.rand(128)/10)
    b.append(np.random.rand()/10)

In [20]:
## 对词汇表进行初始化表征，选取128维向量表征每个词
lookup = list(np.random.rand(len(unique_tokens),128)/10)

In [21]:
## 训练
n = 0
m = 0
while (n<100):
    e_W = 0
    e_b = 0
    
    ## 获取随机输入
    x_context_rep, x_word_rep, x_context, x_word = random_sample(representations,words)
    
    ## 索引哈夫曼树节点和编码
    # 索引编码
    if (x_word in leaves_code.values()):
        x_code = leaves_code[x_word]
    else:
        x_code = leaves_code['_UNK']

    # 索引节点
    x_nodes = []
    x_nodes_index = []
    if ([x for x in leaves_list if x.get_name() == x_word] == []):
        node = [x for x in leaves_list if x.get_name() == '_UNK'][0]
    else:
        node = [x for x in leaves_list if x.get_name() == x_word][0]
    while (node.get_parent()):   
        x_nodes.append(node)
        x_nodes_index.append(leaves_list.index(node))
        node = node.get_parent()
    x_nodes.reverse()
    x_nodes_index.reverse()
    
    ## word embedding
    x_context_embedding = []
    for item in x_context_rep:
        x_context_embedding.append(lookup[item])
    x_word_embedding = lookup[x_word_rep]
    x_w = np.average(x_context_embedding,axis = 0)
    
    
    ## 一次迭代更新
    for j in range(len(x_nodes_index)):
        f = 1/(1+np.exp(-np.matmul(x_w,W[x_nodes_index[j]])-b[x_nodes_index[j]]))
        g = (1- x_code[j] - f)*0.1
        e_W = e_W + g* W[x_nodes_index[j]]
        e_b = e_b + g* b[x_nodes_index[j]]
        dW = W[x_nodes_index[j]] + g * x_w.T
        W[x_nodes_index[j]] = dW
        db = b[x_nodes_index[j]] + g
        b[x_nodes_index[j]] = db
        
    
    for i in range(len(x_context_rep)):
        dlookup = lookup[x_context_rep[i]] + e_W
        lookup[x_context_rep[i]] = dlookup
        
    
    if(np.sum(e_W**2) < 0.000001):
        m += 1
    
    if(m >=10):
        break
    

    n += 1

    
    


节点的索引通过递归实现，需要返回节点在列表中的索引号，之后需要对参数进行索引。反转实现节点从根到叶排列

在CBOW中，沿着左子树走，那么就是负类(霍夫曼树编码1)，沿着右子树走，那么就是正类(霍夫曼树编码0)

输入：训练样本，词向量的维度大小，上下文大小2c,步长
输出：霍夫曼树的内部节点模型参数，所有的词向量w
1. 基于语料训练样本建立霍夫曼树。
2. 随机初始化所有的模型参数θ，所有的词向量w。
3. 进行梯度上升迭代过程，对于训练集中的每一个样本(context(w),w)(context(w),w)做如下处理：
    a)  e=0， 通过2c个context的平均值表征w；
    b)  对于霍夫曼树每层结构进行计算迭代，遵循的原则是保证按照树上编码的路径返回根节点，通过梯度上升法来不断更新分叉上的参数；
    c)  对所有2c个词向量进行更新；
    d)  如果梯度收敛，则结束梯度迭代，否则回到步骤3继续迭代。
参考：http://www.cnblogs.com/pinard/p/7243513.html

In [22]:
## 通过CBOW更新后的embedding矩阵对输入数据进行表征
x_origin = []
for seq in clean_tokenized:
    reps = []
    for word in seq:
        reps.append(lookup[get_index(word)])
    x_origin.append(reps)

In [23]:
## 计算每个句子的长度并存储
lens = []
for words in clean_tokenized:
    lens.append(len(words))
print(max(lens))

36


In [24]:
## 将每个句子补全到最大长度,对于padding的部分通过全0向量来补充
x = []
for reps in x_origin:
    if len(reps) < max(lens):
        for i in range(max(lens)-len(reps)):
            reps.append(np.zeros(128))
    x.append(reps)
x = np.array(x)

In [25]:
## 定义y并且生成one-hot vector
y = []
for item in data_train:
    y.append(item[3])
classes = np.max(y) + 1
y = np.eye(classes)[y]

In [26]:
## 划分集合
n_train = int(len(x)*0.8)
n_test = int(len(x)*0.1)
n_dev = int(len(x) - n_train -n_test)

import random
index_train = random.sample(range(len(x)),n_train)
index_dev = random.sample(list(set(range(len(x)))^set(index_train)) ,n_dev)
index_test = list(set(list(set(range(len(x)))^set(index_train)))^set(index_dev)) 

x_train = []
y_train = []
lens_train = []
for index in index_train:
    x_train.append(x[index])
    y_train.append(y[index])
    lens_train.append(lens[index])

x_dev = []
y_dev = []
lens_dev = []
for index in index_dev:
    x_dev.append(x[index])
    y_dev.append(y[index])
    lens_dev.append(lens[index])

x_test = []
y_test = []
lens_test = []
for index in index_test:
    x_test.append(x[index])
    y_test.append(y[index])
    lens_test.append(lens[index])

x_train = np.array(x_train)
y_train = np.array(y_train)

x_dev = np.array(x_dev)
y_dev = np.array(y_dev)

x_test = np.array(x_test)
y_test = np.array(y_test)

In [133]:
## 定义一个随机函数，随机抽取
import random
def random_sample(x,y,lens_):
    index = random.sample(range(len(x)),1)[0]
    balanced_m = np.concatenate((np.ones((lens_[index], 5),dtype = np.float32),np.zeros((max(lens)-lens_[index], 5),dtype = np.float32)))
    return x[index],y[index],balanced_m

In [147]:
## RNN模型
# 定义输入和输出,以及一个平衡矩阵
x_input = tf.placeholder(tf.float32,[max(lens),128])
y_label = tf.placeholder(tf.float32,[5])
balanced_matrix = tf.placeholder(tf.float32,[max(lens),5])

In [148]:
# 定义每一个输入单元，构建双层RNN结构
cell = tf.nn.rnn_cell.BasicLSTMCell(128,state_is_tuple=True)
multi_cell = tf.nn.rnn_cell.MultiRNNCell([cell]*2,state_is_tuple=True)

In [149]:
# 初始化，每次输入一个数据
state = multi_cell.zero_state(1, tf.float32)

In [150]:
## 建立RNN序列
outputs = []
with tf.variable_scope("RNN"):
    for time_step in range(max(lens)):
        if time_step > 0: 
            tf.get_variable_scope().reuse_variables()
        (cell_output,state) = cell(x_input[time_step, :],state)
        outputs.append(cell_output)

AttributeError: 'LSTMStateTuple' object has no attribute 'get_shape'

In [113]:
x.get_shape()

AttributeError: 'numpy.ndarray' object has no attribute 'get_shape'